In [1]:
import sqlite3
import sys
sys.path.append('../../')
from src.preprocessor.preprocessing import FileIO
from src.llm.llm_interface import LLM
from src.llm.llm_utils import load_azure_openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), override=True)

In [2]:
llm = load_azure_openai()

### Import Metadata

In [10]:
data_dir = '../../data/metadata/'
data = []
names = ['guests', 'titles', 'short_summaries', 'view_counts']
for name in names:
    data.append(tuple(FileIO.load_json(f'{data_dir}{name}.json')))

### Create DB and Ingest Data

In [34]:
file = 'sqlite3.db'

In [35]:
conn= sqlite3.connect(file)

In [36]:
cursor = conn.cursor()

#### Create basic table

In [37]:
table_schema = """
CREATE TABLE IF NOT EXISTS huberman (
    id INTEGER PRIMARY KEY,
    guests TEXT NOT NULL,
    titles TEXT NOT NULL,
    summaries TEXT NOT NULL,
    view_counts INTEGER NOT NULL
);
"""

In [38]:
_ = cursor.execute(table_schema)

In [39]:
_ = cursor.execute('SELECT * from huberman;')

In [40]:
#should be an empty list
result = cursor.fetchall()
print(result)

[]


### Insert Data into DB

In [51]:
def insert_rows(guests, titles, summaries, counts): 
    connection = sqlite3.connect(file)
    cursor = connection.cursor()
    for guest, title, summary, count in zip(guests, titles, summaries, counts):
        insert_query =  f"""
                        INSERT INTO huberman (guests, titles, summaries, view_counts)
                        VALUES (?,?,?,?);
                        """
        cursor.execute(insert_query, (guest, title, summary, count))
    connection.commit()
    cursor.close()
    connection.close()

In [42]:
def get_cursor():
    conn = sqlite3.connect(file)
    cursor = conn.cursor()
    return cursor, conn

In [43]:
def get_response(command: str):
    cursor, conn = get_cursor()
    cursor.execute(command)
    answer = cursor.fetchall()
    cursor.close()
    conn.close()
    return answer

In [44]:
#from data import above
guests, titles, summaries, view_counts = [*data]

In [53]:
insert_rows(guests, titles, summaries, view_counts)

In [54]:
command = 'SELECT * FROM huberman;'

In [57]:
result = get_response(command)

#### Check results are what you expect

In [58]:
id, guest, title, summary, view_count = [*result[0]]

print(f'ID: {id}')
print(f'Guest: {guest}')
print(f'Title: {title}')
print(f'Summary: {summary}')
print(f'View Count: {view_count}')

ID: 1
Guest: Dr. Matthew Walker
Title: Dr. Matthew Walker: The Biology of Sleep & Your Unique Sleep Needs | Huberman Lab Guest Series
Summary: In the Huberman Lab podcast episode featuring Dr. Matthew Walker, the discussion centers on the biology of sleep and how it impacts mental and physical health. They delve into the different stages of sleep, including non-REM and REM sleep, and their unique roles in brain function, such as memory consolidation and emotional regulation. They also explore practical strategies for improving sleep quality, such as managing light exposure, temperature, and timing of activities. Dr. Walker introduces the QQRT (Quality, Quantity, Regularity, and Timing) formula for identifying individual sleep needs. The conversation highlights the detrimental effects of sleep deprivation on hormones, metabolism, and the risk of diseases like diabetes and Alzheimer's. They touch on the evolutionary importance of sleep, suggesting that its universal presence among specie

In [46]:
system_message = """
Your main function is to generate flawless SQL queries from plain text language, that will enable a user 
to query a SQLite3 database.  There is one table called huberman in the database with four columns labeled:
1. guests
2. titles
3. summaries
4. view_counts
"""

In [65]:
base_user_message = """
Convert the plain text query into a SQL query that can answer questions about the data contained in the huberman table. 
Output your response in JSON format, do not include any newlines, and return only the SQL command nothing else.
--------------
QUERY: {query}
--------------
SQL:
"""

In [94]:
response_system_message = """
You excel at answering questions from a multitude of information sources.  Specifically you know how to translate SQL-generated answers into actionable insights
"""

In [174]:
base_response_user_message = """
You will be given a user question and a response to that query pulled from a SQL database.  
Use the information from the response to answer the original query.  Answer the question in an objective tone, and do not make any reference to the SQL database.
If the information in the answer is presented in a list, ensure that you return your output in the same order, do not mix up the original ordering. 
If a specific piece of text information is requested such as a guest name or show title, use the LIKE operator and % wildcard character as appropriate to address the user information need.
If the response does not answer the question, then either state that you do not know the answer, or answer the question based on prior knowledge.
----------------
USER QUERY: {query}
----------------
ANSWER FROM SQL DATABASE: {sql_answer}
----------------
OUTPUT: 
"""

In [183]:
query = 'Return show titles that discuss muscle hypertrophy with over 500K views'
user_message = base_user_message.format(query=query)
response = llm.chat_completion(system_message, user_message)
answer = get_response(response)
post_user_message = base_response_user_message.format(query=query, sql_answer=answer)
post_response = llm.chat_completion(response_system_message, post_user_message)

In [184]:
print(response)
print(answer)
print(post_response)

SELECT titles 
FROM huberman 
WHERE summaries LIKE '%muscle hypertrophy%' 
AND view_counts > 500000;
[('Dr. Andy Galpin: How to Assess & Improve All Aspects of Your Fitness | Huberman Lab Guest Series',), ('Science of Muscle Growth, Increasing Strength & Muscular Recovery',)]
The show titles that discuss muscle hypertrophy with over 500K views are:
1. Dr. Andy Galpin: How to Assess & Improve All Aspects of Your Fitness | Huberman Lab Guest Series
2. Science of Muscle Growth, Increasing Strength & Muscular Recovery


In [129]:
print(response)

SELECT titles 
FROM huberman 
WHERE summaries LIKE '%mental health%' 
AND view_counts > 1000000


In [130]:
conn=sqlite3.connect(file)
cur = conn.cursor()

In [158]:
cur.execute("SELECT titles FROM huberman WHERE summaries LIKE '%mental health%' AND view_counts > 100000")

In [159]:
new_answer = cur.fetchall()
new_answer

[('LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Melbourne, AU',),
 ('How Placebo Effects Work to Change Our Biology & Psychology',),
 ('How to Improve Oral Health & Its Critical Role in Brain & Body Health',),
 ('Dr. Kay Tye: The Biology of Social Interactions and Emotions',),
 ('Journal Club with Dr. Peter Attia | Effects of Light & Dark on Mental Health & Treatments for Cancer',),
 ('LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Toronto, ON',),
 ('AMA #12: Thoughts on Longevity Supplements (Resveratrol, NR, NMN, Etc.) & How to Improve Memory',),
 ('Mental Health Toolkit: Tools to Bolster Your Mood & Mental Health',),
 ('Mark Zuckerberg & Dr. Priscilla Chan: Curing All Human Diseases & the Future of Health & Technology',),
 ('Dr. Paul Conti: Tools and Protocols for Mental Health | Huberman Lab Guest Series',),
 ('Dr. Paul Conti: How to Improve Your Mental Health | Huberman Lab Guest Series',),
 ('Dr. Paul Conti: How to Understand & Assess Your Mental Health | Hu

In [161]:
post_user_message = base_response_user_message.format(query=query, sql_answer=new_answer)
post_response = llm.chat_completion(response_system_message, post_user_message, max_tokens=1500)
print(post_response)

Based on the information provided, the show titles that discussed mental health and had over 100K views are as follows:

1. LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Melbourne, AU
2. How Placebo Effects Work to Change Our Biology & Psychology
3. How to Improve Oral Health & Its Critical Role in Brain & Body Health
4. Dr. Kay Tye: The Biology of Social Interactions and Emotions
5. Journal Club with Dr. Peter Attia | Effects of Light & Dark on Mental Health & Treatments for Cancer
6. LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Toronto, ON
7. AMA #12: Thoughts on Longevity Supplements (Resveratrol, NR, NMN, Etc.) & How to Improve Memory
8. Mental Health Toolkit: Tools to Bolster Your Mood & Mental Health
9. Mark Zuckerberg & Dr. Priscilla Chan: Curing All Human Diseases & the Future of Health & Technology
10. Dr. Paul Conti: Tools and Protocols for Mental Health | Huberman Lab Guest Series
11. Dr. Paul Conti: How to Improve Your Mental Health | Huberman Lab Gue

In [62]:
object.__class__

type

In [64]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [65]:
vars(object)

mappingproxy({'__new__': <function object.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'object' objects>,
              '__hash__': <slot wrapper '__hash__' of 'object' objects>,
              '__str__': <slot wrapper '__str__' of 'object' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'object' objects>,
              '__setattr__': <slot wrapper '__setattr__' of 'object' objects>,
              '__delattr__': <slot wrapper '__delattr__' of 'object' objects>,
              '__lt__': <slot wrapper '__lt__' of 'object' objects>,
              '__le__': <slot wrapper '__le__' of 'object' objects>,
              '__eq__': <slot wrapper '__eq__' of 'object' objects>,
              '__ne__': <slot wrapper '__ne__' of 'object' objects>,
              '__gt__': <slot wrapper '__gt__' of 'object' objects>,
              '__ge__': <slot wrapper '__ge__' of 'object' objects>,
              '__init__': <slot wrapper '__init__' of

Bad pipe message: %s [b"\x04\x04G\x98~\xa6\xd3U\xb7\x11\x08\xa1\x90W\xc8\x04Gd \xc9c\x95\xb1%eE\xf9\xde\x967\xe3=\x94i\x0b\xb9\xcfJ\x90%\xc7\xff'\xb3x\xb3\x91\xfed\xab\xcf\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$"]
Bad pipe message: %s [b'\xbb\x87\x8c,\x0f\xb73z\x17\x9e\x03\x96\x7f\xd4\x16\xabVJ \x1f{\xe4\x9fe', b'\xd2~ %\xfd\xd2HI\xf6\x94\xd9\x10+\xc2\x9f\x13\xaa2D\x18\xdf\xd6\xf8\x16\x88\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x